# Motivating example

In [ ]:
import numpy as np
import numba

In [ ]:
def mean_python(a):
    total = 0
    for i in a:
        total += i
    return total / len(a)

def mean_numpy(a):
    return np.mean(a)

mean_numba = numba.jit(mean_python)

In [ ]:
a = np.random.randint(1, 100, 1000000)

In [ ]:
%time mean_python(a)
%time mean_numpy(a)
%time mean_numba(a)

In [ ]:
%timeit mean_python(a)
%timeit mean_numpy(a)
%timeit mean_numba(a)